In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain
from langchain import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
import sys

class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "GLM"

    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True).half().cuda()

    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

    
modelpath = "/root/model/chatglm-6b"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

file = 'clothing-short.csv'
loader = CSVLoader(file_path=file)

import pandas as pd
data = pd.read_csv(file,header=None)
data

In [3]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

from IPython.display import display, Markdown 
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
data = loader.load()
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

db = FAISS.from_documents(data, embeddings)
docs = db.similarity_search(query)


db = DocArrayInMemorySearch.from_documents(docs, embeddings)


,0,1,2
0,NaN,name,description
1,0.0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
2,1.0,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
3,2.0,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
4,3.0,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
...,...,...,...
996,995.0,"Men's Classic Denim, Standard Fit",Crafted from premium denim that will last wash...
997,996.0,CozyPrint Sweater Fleece Pullover,The ultimate sweater fleece - made from superi...
998,997.0,Women's NRS Endurance Spray Paddling Pants,These comfortable and affordable splash paddli...
999,998.0,Women's Stop Flies Hoodie,This great-looking hoodie uses No Fly Zone Tec...


In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


with open("pirlo.txt") as f:
    book = f.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(book)
docsearch = Chroma.from_texts(
    texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]
)


query = "皮尔洛是谁?"
docs = docsearch.similarity_search_with_score(query, k=1)
for doc in docs:
    print(doc)



ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)